In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier

/home/janardhan/anaconda3/envs/dl/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [1]:
# loading validations predictions for ensembling

In [2]:
filenames = np.load('filenames_list.npy')

In [290]:
resnet_preds = np.load('resnet_preds.npy')
mobilenet_preds = np.load('mobilenet_preds.npy')
cacnn_preds = np.load('cacnn_preds.npy')

In [291]:
resnet_preds_224 = np.load('resnet_preds_224.npy')
mobilenet_preds_224 = np.load('mobilenet_preds_224.npy')
cacnn_preds_224 = np.load('cacnn_preds_224.npy')

In [292]:
resnet_preds_classes = np.argmax(resnet_preds,axis=1)
mobilenet_preds_classes = np.argmax(mobilenet_preds,axis=1)
cacnn_preds_classes = np.argmax(cacnn_preds,axis=1)

In [293]:
resnet_preds_classes_224 = np.argmax(resnet_preds_224,axis=1)
mobilenet_preds_classes_224 = np.argmax(mobilenet_preds_224,axis=1)
cacnn_preds_classes_224 = np.argmax(cacnn_preds_224,axis=1)

In [294]:
mean_preds = (resnet_preds+mobilenet_preds+cacnn_preds+resnet_preds_224+mobilenet_preds_224+cacnn_preds_224)/6

In [295]:
np.bincount(np.argmax(mean_preds,axis=1)==target_classes)

array([ 24, 850])

In [296]:
feature_arr_classes = np.array([mobilenet_preds_classes,resnet_preds_classes,cacnn_preds_classes,mobilenet_preds_classes_224,resnet_preds_classes_224,cacnn_preds_classes_224])

In [297]:
max_voting = [np.argmax(np.bincount(feature_arr_classes[:,i])) for i in range(874)]

In [298]:
np.bincount(max_voting==target_classes)

array([ 24, 850])

In [15]:
cnt=0
for i in range(len(resnet_preds_classes)):
    if(mobilenet_preds_classes_224[i]!=mobilenet_preds_classes[i]):
        cnt = cnt +1

In [16]:
cacnn_preds_classes.shape

(874,)

In [17]:
target_classes = np.load('target_classes.npy')

In [18]:
target_classes_one_hot = to_categorical(target_classes)

In [19]:
feature_arr = np.concatenate([mobilenet_preds,resnet_preds,cacnn_preds,mobilenet_preds_224,resnet_preds_224,cacnn_preds_224],axis=1)

In [21]:
np.bincount(mobilenet_preds_classes==target_classes)

array([ 20, 854])

In [22]:
features_df = pd.DataFrame(feature_arr)

In [2]:
# splitting the data into train test split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(features_df, target_classes, test_size = 0.2, random_state = 0)

In [25]:
print(X_train.shape[0])
print(X_test.shape[0])

699
175


In [3]:
# ensembling with Random Forest

In [26]:
model_rf = RandomForestClassifier(n_estimators=25,verbose=0,oob_score=True)

In [27]:
parameters_rf = {'random_state':[10,20,30,40,50],'n_estimators':[2,4,7,10,20,40,80,100,150],'criterion':['gini','entropy'],'max_features':['auto','log2'],'class_weight':['balanced',None],'warm_start':['True','False']}

In [28]:
grid_obj_rf = GridSearchCV(estimator= model_rf, param_grid= parameters_rf, scoring= 'accuracy',refit=True,cv=3,n_jobs=6)

In [ ]:
grid_obj_rf.fit(X_train, y_train)

In [30]:
grid_obj_rf.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_features': 'log2',
 'n_estimators': 20,
 'random_state': 40,
 'warm_start': 'True'}

In [31]:
accuracy_score(grid_obj_rf.best_estimator_.predict(X_test),y_test)

0.98285714285714287

In [32]:
grid_obj_rf.best_score_

0.98426323319027187

In [218]:
model_rf1 = RandomForestClassifier(n_estimators=35,verbose=0,oob_score=True,random_state=40)
model_rf1.fit(features_df,target_classes)
model_rf1.oob_score_

0.98398169336384445

In [67]:
#logistic regression

In [68]:
model_lr = LogisticRegression(verbose=0,class_weight='balanced')

In [99]:
parameters_lr = {'random_state':[10,20,30,40,50],'C':[1.0,0.7,0.5,0.9],'solver':['newton-cg', 'sag', 'saga','lbfgs'],'multi_class':['ovr', 'multinomial'],'warm_start':[True,False]}

In [100]:
grid_obj_lr = GridSearchCV(estimator= model_lr, param_grid= parameters_lr, scoring= 'accuracy',refit=True,cv=3,n_jobs=6)

In [ ]:
grid_obj_lr.fit(X_train, y_train)

In [102]:
grid_obj_lr.best_params_

{'C': 0.5,
 'multi_class': 'multinomial',
 'random_state': 10,
 'solver': 'newton-cg',
 'warm_start': True}

In [103]:
accuracy_score(grid_obj_lr.best_estimator_.predict(X_test),y_test)

0.98285714285714287

In [104]:
grid_obj_lr.best_score_

0.98140200286123036

In [75]:
#decision tree

In [76]:
model_dt = DecisionTreeClassifier(class_weight='balanced')

In [105]:
parameters_dt = {'random_state':[10,20,30,40,50],'criterion':['gini','entropy'],'max_features':['auto','log2'],'min_samples_split':[2,5,7,10,13,15,20]}

In [106]:
grid_obj_dt = GridSearchCV(estimator= model_dt, param_grid= parameters_dt, scoring= 'accuracy',refit=True,cv=3,n_jobs=6)

In [107]:
grid_obj_dt.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'log2'], 'random_state': [10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 7, 10, 13, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [108]:
grid_obj_dt.best_params_

{'criterion': 'entropy',
 'max_features': 'auto',
 'min_samples_split': 2,
 'random_state': 30}

In [109]:
accuracy_score(grid_obj_dt.best_estimator_.predict(X_test),y_test)

0.96571428571428575

In [110]:
grid_obj_dt.best_score_

0.97424892703862664

In [111]:
#SVM

In [112]:
model_svm = SVC(class_weight='balanced',probability=True)

In [113]:
parameters_svm = {'random_state':[10,20,30,40,50],'C':[1.0,0.9,0.8,0.7,0.6,0.5,0.4],'kernel':['rbf','sigmoid','linear','poly']}

In [114]:
grid_obj_svm = GridSearchCV(estimator= model_svm, param_grid= parameters_svm, scoring= 'accuracy',refit=True,cv=3,n_jobs=6)

In [115]:
grid_obj_svm.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'kernel': ['rbf', 'sigmoid', 'linear', 'poly'], 'random_state': [10, 20, 30, 40, 50], 'C': [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [116]:
grid_obj_svm.best_params_

{'C': 0.5, 'kernel': 'linear', 'random_state': 10}

In [117]:
accuracy_score(grid_obj_svm.best_estimator_.predict(X_test),y_test)

0.98285714285714287

In [118]:
grid_obj_svm.best_score_

0.98426323319027187

In [184]:
#xgboost

In [92]:
model_xgb = XGBClassifier(class_weight='balanced',objective='multi:softmax')

In [119]:
parameters_xgb = {'random_state':[10,20,30,40,50],'max_depth':[1,2,3,5],'learning_rate':[0.1,0.3,0.5],'n_estimators':[5,7,10,30,60,100,140],'booster':['gbtree', 'gblinear','dart']}

In [120]:
grid_obj_xgb = GridSearchCV(estimator= model_xgb, param_grid= parameters_xgb, scoring= 'accuracy',refit=True,cv=3,n_jobs=6)

In [121]:
grid_obj_xgb.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
       colsample_bylevel=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='multi:softmax',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'max_depth': [1, 2, 3, 5], 'random_state': [10, 20, 30, 40, 50], 'learning_rate': [0.1, 0.3, 0.5], 'n_estimators': [5, 7, 10, 30, 60, 100, 140], 'booster': ['gbtree', 'gblinear', 'dart']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [122]:
grid_obj_xgb.best_params_

{'booster': 'gblinear',
 'learning_rate': 0.1,
 'max_depth': 1,
 'n_estimators': 60,
 'random_state': 10}

In [123]:
accuracy_score(grid_obj_xgb.best_estimator_.predict(X_test),y_test)

0.98285714285714287

In [124]:
grid_obj_xgb.best_score_

0.98140200286123036

In [125]:
#mean, max-voting

In [143]:
valid_preds_rf = grid_obj_rf.best_estimator_.predict_proba(features_df)
valid_preds_lr = grid_obj_lr.best_estimator_.predict_proba(features_df)
valid_preds_dt = grid_obj_dt.best_estimator_.predict_proba(features_df)
valid_preds_svm = grid_obj_svm.best_estimator_.predict_proba(features_df)
valid_preds_xgb = grid_obj_xgb.best_estimator_.predict_proba(features_df)

In [144]:
valid_preds_mean = (valid_preds_rf+valid_preds_lr+valid_preds_dt+valid_preds_svm+valid_preds_xgb)/5

In [145]:
np.bincount(np.argmax(valid_preds_mean,axis=1)==target_classes)

array([  3, 871])

In [149]:
valid_preds_voting = np.array([np.argmax(valid_preds_rf,axis=1),np.argmax(valid_preds_lr,axis=1),np.argmax(valid_preds_dt,axis=1),np.argmax(valid_preds_svm,axis=1),np.argmax(valid_preds_xgb,axis=1)])

In [150]:
valid_preds_max_voting = [np.argmax(np.bincount(valid_preds_voting[:,i])) for i in range(874)]

In [151]:
np.bincount(valid_preds_max_voting==target_classes)

array([  8, 866])

In [4]:
#ensembling with mean, and voting

In [235]:
mobilenet_test_preds = np.load('mobilenet_test_preds.npy')
resnet_test_preds = np.load('resnet_test_preds.npy')
cacnn_test_preds = np.load('cacnn_test_preds.npy')
mobilenet_test_preds_224 = np.load('mobilenet_test_preds_224.npy')
resnet_test_preds_224 = np.load('resnet_test_preds_224.npy')
cacnn_test_preds_224 = np.load('cacnn_test_preds_224.npy')

In [236]:
test_filenames = np.load('test_filenames.npy')

In [237]:
test_preds = np.concatenate([mobilenet_test_preds,resnet_test_preds,cacnn_test_preds,mobilenet_test_preds_224,resnet_test_preds_224,cacnn_test_preds_224],axis=1)

In [174]:
test_features_df = pd.DataFrame(test_preds)

In [276]:
test_preds_mean = []

In [277]:
for i in range(len(test_filenames)):
    test_preds_mean.append(np.mean(np.array([mobilenet_test_preds[i],resnet_test_preds[i],cacnn_test_preds[i],mobilenet_test_preds_224[i],resnet_test_preds_224[i],cacnn_test_preds_224[i]]),axis=0))

In [203]:
test_preds_rf = grid_obj_rf.best_estimator_.predict_proba(test_features_df)
test_preds_lr = grid_obj_lr.best_estimator_.predict_proba(test_features_df)
test_preds_dt = grid_obj_dt.best_estimator_.predict_proba(test_features_df)
test_preds_svm = grid_obj_svm.best_estimator_.predict_proba(test_features_df)
test_preds_xgb = grid_obj_xgb.best_estimator_.predict_proba(test_features_df)

In [204]:
test_preds_mean = (test_preds_rf+test_preds_lr+test_preds_dt+test_preds_svm+test_preds_xgb)/5

In [205]:
test_preds_voting = np.array([np.argmax(test_preds_rf,axis=1),np.argmax(test_preds_lr,axis=1),np.argmax(test_preds_dt,axis=1),np.argmax(test_preds_svm,axis=1),np.argmax(test_preds_xgb,axis=1)])

In [206]:
test_preds_max_voting = [np.argmax(np.bincount(test_preds_voting[:,i])) for i in range(len(test_filenames))]

In [220]:
final_test_preds = model_rf1.predict(test_preds)

In [279]:
class_map = {'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [280]:
class_map = {class_map[i]:i for i in class_map.keys()}

In [281]:
class_map

{0: 'HTC-1-M7',
 1: 'LG-Nexus-5x',
 2: 'Motorola-Droid-Maxx',
 3: 'Motorola-Nexus-6',
 4: 'Motorola-X',
 5: 'Samsung-Galaxy-Note3',
 6: 'Samsung-Galaxy-S4',
 7: 'Sony-NEX-7',
 8: 'iPhone-4s',
 9: 'iPhone-6'}

In [283]:
final_test_preds = np.argmax(test_preds_mean,axis=1)

In [211]:
len(test_preds_max_voting)

2640

In [284]:
sub_preds = [class_map[i] for i in final_test_preds]

In [285]:
test_filenames.shape

(2640,)

In [5]:
# submitting the final ensembled results

In [286]:
for index,i in enumerate(test_filenames):
    if(i.split('_')[-1]=='manip.tif'):
        sub_preds[index]=''

In [288]:
submission = pd.DataFrame({'fname':test_filenames,'camera':sub_preds})
submission.to_csv('submission_ensemble_mean_only_unalt.csv', encoding="utf8", index=False)

In [289]:
from IPython.display import FileLink
FileLink('submission_ensemble_mean_only_unalt.csv')

/home/janardhan/Documents/submission_ensemble_mean_only_unalt.csv

In [1]:
# Private LB score - 
# mean - 97.39